In [ ]:
%load_ext autoreload
%autoreload 2
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
from pandas import Timedelta
# from optiml.utils import sf
import time
from optiml.utils.sf import logger, sql_to_df, run_sql, conn, session
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

import plotly
plotly.offline.init_notebook_mode()

In [ ]:
conn = session._conn._conn

In [ ]:
session.deepcopy()

In [ ]:
import psycopg

In [ ]:
def exec_query(conn, q):
    with conn.cursor() as c:
        return exec_query_cursor(c,q)

def exec_query_cursor(cursor, q):
    print(q)
    try:
        cursor.execute_async(q)
        qid = cursor.sfqid
        cursor.get_results_from_sfqid(qid)
        results = cursor.fetchall()
        print(results)
        # all_results[q] = results
    except Exception as e:
        # all_results[q] = e


In [ ]:

# Python program to illustrate the concept
# of threading
# importing the threading module
import threading
 
 
def print_cube(num):
    # function to print cube of given num
    print("Cube: {}" .format(num * num * num))
 
 
def print_square(num):
    # function to print square of given num
    print("Square: {}" .format(num * num))
 
 
all_results = {}

# exec_query(conn, 3)
    # creating thread
numthreads = 10
threads = []

cursor = conn.cursor()
for i in range(numthreads):
    print(i)
    # threads.append(threading.Thread(target=print_square, args=(i,)))
    q = f"select {i}"
    threads.append(threading.Thread(target=exec_query, args=(conn,q)))
    
    if i == 4:
        q2 = "bad statement"
    else:
        q2 = f"select 'second {i}'"
    threads.append(threading.Thread(target=exec_query, args=(conn,q2)))
    # threads.append(threading.Thread(target=exec_query_cursor, args=(cursor,i)))
 
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
    
#     # starting thread 1
#     t1.start()
#     # starting thread 2
#     t2.start()
 
#     # wait until thread 1 is completely executed
#     t1.join()
#     # wait until thread 2 is completely executed
#     t2.join()
 
#     # both threads completely executed
#     print("Done!")

In [ ]:
all_results

In [ ]:
def exec_query(conn, q):
    with conn.cursor() as c:
        return exec_query_cursor(c,q)

def exec_query_cursor(cursor, q):
    print(f"running query {q}")
    # try:
    cursor.execute_async(q)
    qid = cursor.sfqid
    cursor.get_results_from_sfqid(qid)
    results = cursor.fetchall()
    return results
        # all_results[q] = results
    # except Exception as e:
    #     return e
        # all_results[q] = e


In [ ]:
import concurrent.futures

# Define your graph of functions and dependencies

import time

# def task(n):
#     print(f"performing task {n}")
#     if n == 2:
#         time.sleep(2)
#     if n == 2:
#         raise ValueError("wtf")
#     return n

# graph = {
#     'node1': {
#         'dependencies': [], 
#         'fn': exec_query,
#         'args': (conn, "select 1",)
#     },
#     'node2': {
#         'dependencies': [], 
#         'fn': exec_query,
#         'args': (conn, "select 2",)
#     },
#     'node3': {
#         'dependencies': ['node2', 'node1'], 
#         'fn': exec_query,
#         'args': (conn, "select 3",)
#     },
#     'node4': {
#         'dependencies': ['node1'], 
#         'fn': exec_query,
#         'args': (conn, "select 4",)
#     }
# }

graph = {
    'node1': {
        'dependencies': [], 
        'fn': exec_query,
        'args': (conn, "create or replace table a as select 1 col",)
    },
    'node2': {
        'dependencies': ['node1'], 
        'fn': exec_query,
        'args': (conn, "create or replace table b as select * from a",)
    },
    'node3': {
        'dependencies': ['node2', 'node1'], 
        'fn': exec_query,
        'args': (conn, "create or replace table c as select * from a union select * from b",)
    },
    'node4': {
        'dependencies': ['node1'], 
        'fn': exec_query,
        'args': (conn, "create or replace table d as select * from a",)
    }
}
import threading
lock = threading.Lock()

# Create a dictionary to store the results of each function
results = {}
errors = {}
def run_function(node):
    # Check if all dependencies have been completed
    # dependencies = graph[node]['dependencies']
    # for dependency in dependencies:
    #     if dependency not in results:
    #         raise ValueError(f"Dependency {dependency} of node {node} has not been completed")

    # Execute the function and store the result
    try:
        task_result = graph[node]['fn'](*graph[node]['args'])
        with lock:
            results[node] = task_result
            print(f"{node} complete; task_result: {task_result}")

    except Exception as e:
        task_result = e
        with lock:
            errors[node] = e

import queue
task_queue = queue.Queue()

# Add the initial tasks to the queue
for node in graph:
    task_queue.put(node)

# Set the maximum number of threads
max_threads = 1

# Create a ThreadPoolExecutor with the maximum number of threads
with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
    # Create a dictionary to store the futures
    futures = {}

    # Process the tasks from the queue
    while not task_queue.empty():
        node = task_queue.get()
        # print(f"evaluating node {node}")
        # Check if all dependencies have been completed
        dependencies = graph[node]['dependencies']
        dependencies_completed = all(dependency in results for dependency in dependencies)
        
        dependencies_errored = any(dependency in errors for dependency in dependencies)
        # print(f"dependencies complete: {dependencies_completed}")
        if dependencies_completed:
            print(f"dependencies complete for node {node}. Submitting.")
            # Submit the function to the executor
            future = executor.submit(run_function, node)
            futures[node] = future

            # # Add the dependent tasks to the queue
            # for dependent_node in graph:
            #     if node in graph[dependent_node]['dependencies']:
            #         task_queue.put(dependent_node)
        elif dependencies_errored:
            raise ValueError(f"dependencies for {node} errored")
        else:
            # print(f"dependencies not complete for node {node}. Submitting.")
            # Re-add the task to the queue for later processing
            task_queue.put(node)

print("waiting for completion...")
# Wait for all futures to complete
concurrent.futures.wait(futures.values())

# Access the results
for node, future in futures.items():
    result = future.result()
    print(f"Result of {node}: {result}")

In [ ]:
errors